Importing the dependencies

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



Device Configuration

In [ ]:
# check for CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [ ]:
# Data collection and preprocessing
data = load_breast_cancer()
X, y = data.data, data.target

In [ ]:
print(X)

[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]


In [ ]:
print(y[:5])

[0 0 0 0 0]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(569, 30)
(455, 30)
(114, 30)


In [ ]:
# standardise data using standard scalar
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
type(X_train)

numpy.ndarray

In [ ]:
# convert data to pytorch tensors
X_train = torch.tensor(X_train, dtype = torch.float32).to(device)
y_train = torch.tensor(y_train, dtype = torch.float32).to(device)
X_test = torch.tensor(X_test, dtype = torch.float32).to(device)
y_test = torch.tensor(y_test, dtype = torch.float32).to(device)



In [ ]:
# neural network
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(NeuralNet, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size) # fully connected layer 1
    self.relu = nn.ReLU() # non linearity
    self.fc2 = nn.Linear(hidden_size, output_size) # fully connected hidden layer 2
    self.sigmoid = nn.Sigmoid()
# passing the data through neural network layers
  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out) # for handling all dtypes of numbers
    out = self.fc2(out)
    out = self.sigmoid(out)
    return out


In [ ]:
# define hyperparameters
input_size = X_train.shape[1] # 30
hidden_size = 64
output_size = 1 # sigmoid
learning_rate = 0.001
num_epochs = 100

In [ ]:
#initialize neural network and move it to gpu
model = NeuralNet(input_size, hidden_size, output_size).to(device)


In [ ]:
# define loss and optimizer
criterion = nn.BCELoss()
# adam optimizer
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
# training the NN
for epoch in range(num_epochs):
  model.train()
  optimizer.zero_grad()
  outputs = model(X_train)
  loss = criterion(outputs, y_train.view(-1,1))
  loss.backward()
  optimizer.step()

  # calculate accuracy
  with torch.no_grad():
    predicted = outputs.round()
    correct = (predicted == y_train.view(-1,1)).float().sum()
    accuracy = correct / y_train.size(0)
  if (epoch+1) % 10 == 0:
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss : {loss.item():.4f}, Accuracy: {accuracy.item()*100:.2f}%")



Epoch [10/100], Loss : 0.6020, Accuracy: 75.16%
Epoch [20/100], Loss : 0.4675, Accuracy: 93.63%
Epoch [30/100], Loss : 0.3609, Accuracy: 94.95%
Epoch [40/100], Loss : 0.2812, Accuracy: 95.60%
Epoch [50/100], Loss : 0.2248, Accuracy: 96.04%
Epoch [60/100], Loss : 0.1860, Accuracy: 96.70%
Epoch [70/100], Loss : 0.1590, Accuracy: 97.14%
Epoch [80/100], Loss : 0.1397, Accuracy: 97.36%
Epoch [90/100], Loss : 0.1253, Accuracy: 97.58%
Epoch [100/100], Loss : 0.1141, Accuracy: 98.02%


In [ ]:
# model evaluation
model.eval()
with torch.no_grad():
  outputs = model(X_train)
  predicted = outputs.round()
  correct = (predicted == y_train.view(-1,1)).float().sum()
  accuracy = correct / y_train.size(0)
  print(f"accuracy on training data: {accuracy.item()* 100:.2f}%")

accuracy on training data: 98.02%


In [ ]:
# model evaluation
model.eval()
with torch.no_grad():
  outputs = model(X_test)
  predicted = outputs.round()
  correct = (predicted == y_test.view(-1,1)).float().sum()
  accuracy = correct / y_test.size(0)
  print(f"accuracy on test data: {accuracy.item()* 100:.2f}%")

accuracy on test data: 97.37%
